### Преамбула

В этом ноутбуке задокументированы манипуляции с данными, которые мы проделали при обработке файлов судебной статистики с данными за 2009 - 2018 года. Работа кода демонстрируется на файлах статистики 2018 года, однако если подставить вместо него любой другой год, начиная с 2009, - все будет работать. В самом конце приведен код для обработки данных за все доступные года и сведения их в одну таблицу. Все написанные нами функции вынесены в отдельный файл: tools.py.

### Оглавление

- [Источник данных](#data_source)
- [Переименование файлов](#rename_files)
- [Обработка файлов формы 10.3](#10.3)
- [Обработка файлов формы 10.3.1](#10.3.1)
- [Обработка файлов формы 10-а](#10-a)
- [Проверка корректности обработки данных](#data_check)
- [Сведение обработанных таблиц в одну длинную](#final_table)
- [Обработка файлов за весь доступный период](#all)

<a id='data_source'></a>
### Источник данных

Уголовную статистику берем с [сайта Судебного департамента](http://www.cdep.ru/index.php?id=79) при Верховном суде РФ. Для обработки мы выбрали два типа данных: статистику по количеству осужденных по основному составу преступления (без учета сложения наказаний) и по дополнительному.

Данные по основным составам за 2009-2010 года находятся в одной таблице:

*№ 10.3 Виды наказания по наиболее тяжкому преступлению (без учета сложения)*,

за 2011-2018 года в двух таблицах:

*№ 10.3 «Отчет о видах наказания по наиболее тяжкому преступлению (без учета сложения)»*

*№ 10.3.1 «Отчет о сроках лишения свободы и размерах штрафов (приложение к отчету формы № 10.3)»*

Статистика по осужденным по дополнительным составам:

*№ 10-а «Отчет о числе осужденных по всем составам преступлений Уголовного кодекса Российской Федерации»*

<a id='rename_files'></a>
### Файлы

Статистические формы Судебного департамента для обработки лежат в папке ExcelFiles, каждая форма в своей папке:

Main: № 10.3 Виды наказания по наиболее тяжкому преступлению (без учета сложения)

MainParameters: № 10.3.1 «Отчет о сроках лишения свободы и размерах штрафов (приложение к отчету формы № 10.3)

Add: № 10-а «Отчет о числе осужденных по всем составам преступлений Уголовного кодекса Российской Федерации»

Файлы переименованы единообразно: год + имя папки, например: 2009main.xls

Код для переименовывания - в файле rename_files.ipynb

In [1]:
import os
import re
import pandas as pd
import numpy as np
from natsort import natsorted, index_natsorted, order_by_index
from tools import * # функции для обработки
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)

<a id='10.3'></a>

### Обработка файлов формы № 10.3

Для примера возьмем файл за 2018 год.

In [2]:
year = 2018

Первый лист .xls-файла мы пропускаем, так как во всех отчетных формах там содержится общая информация, а не данные.

In [3]:
mailFile = 'ExcelFiles/Main/' + str(year) + 'main.xls'
mainDF = pd.read_excel(mailFile, 1, header=None)

In [4]:
mainDF.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,"Наименование организации, представившей отчет",NaN,NaN,NaN,Судебный департамент при Верховном Суде Россий...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Форма № 10.3,NaN,NaN
1,Категория суда,NaN,NaN,NaN,все суды,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Категория дел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Раздел 1. ВИДЫ НАКАЗАНИЯ ПО НАИБОЛЕЕ ТЯЖКОМУ П...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Статьи УК РФ,№ п/п,ВСЕГО ОСУЖДЕНО,ВИДЫ основного НАКАЗАНИЯ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,По приговору освобождено осужденных от наказания,NaN,NaN,Оправдано,ПРЕКРАЩЕНО,NaN,NaN,NaN,NaN,NaN,NaN,Принудительные меры к невменяемым,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ,NaN,NaN,NaN,Преступление не является оконченным \n(пригото...,"Деяние совершено при обстоятельствах, исключаю...",NaN,NaN
5,NaN,NaN,NaN,пожизненное лишение свободы,лишение свободы,условное осуждение к лишению свободы,содержание в дисциплинарной воинской части,арест,ограничение свободы,ограничение по военной службе,исправительные работы,обязательные работы,принудительные работы,лишение права занимать определенные должности ...,штраф,условное осуждение к иным мерам,по амнистии,по другим основаниям:,NaN,NaN,"за отсутствием события, состава, непричастност...",по амнистии,за примирением с потерпевшим \n(ст. 25 УПК РФ),в связи с деятельным раскаянием \n(ст. 28 УПК...,назначена мера уголовно-правового характера су...,по другим основаниям,из графы 22,NaN,лишение права занимать определенные должности ...,штраф,"лишение специального, воинского или почетного ...",ограничение свободы,NaN,необходимая оборона \n(ст. 37 УК РФ),крайняя необходимость\n(ст. 39 УК РФ),"обстоятельства, предусмотренные \nстатьями 38,..."
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,от лишения свободы,от иных мер или наказание не назначалось,NaN,NaN,NaN,NaN,NaN,NaN,NaN,на основании примечаний к статьям УК РФ (в свя...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,А,Б,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
8,Всего по составам УК РФ,1,658291,59,187794,169030,90,5,22966,187,56450,116058,795,183,85760,12629,1026,1370,3889,1446,2102,250,125866,18171,33329,10468,6182,8171,57475,7852,171,7080,38574,6,0,0
9,105 ч. 1,2,5723,0,5683,28,0,0,1,0,0,0,0,0,0,0,0,9,2,19,1,0,0,0,0,59,0,448,6,0,3,532,834,1,0,0


После этого удаляем "шапку" таблицы со служебной информацией, колонки, не содержащие номера статьи или значения показателя (пустые и колонки "№ п/п" и "№ стр.")

In [5]:
mainDF = dropFirstRows(mainDF, 'Статьи УК РФ')

In [6]:
mainDF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,Статьи УК РФ,№ п/п,ВСЕГО ОСУЖДЕНО,ВИДЫ основного НАКАЗАНИЯ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,По приговору освобождено осужденных от наказания,NaN,NaN,Оправдано,ПРЕКРАЩЕНО,NaN,NaN,NaN,NaN,NaN,NaN,Принудительные меры к невменяемым,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ,NaN,NaN,NaN,Преступление не является оконченным \n(пригото...,"Деяние совершено при обстоятельствах, исключаю...",NaN,NaN
1,NaN,NaN,NaN,пожизненное лишение свободы,лишение свободы,условное осуждение к лишению свободы,содержание в дисциплинарной воинской части,арест,ограничение свободы,ограничение по военной службе,исправительные работы,обязательные работы,принудительные работы,лишение права занимать определенные должности ...,штраф,условное осуждение к иным мерам,по амнистии,по другим основаниям:,NaN,NaN,"за отсутствием события, состава, непричастност...",по амнистии,за примирением с потерпевшим \n(ст. 25 УПК РФ),в связи с деятельным раскаянием \n(ст. 28 УПК...,назначена мера уголовно-правового характера су...,по другим основаниям,из графы 22,NaN,лишение права занимать определенные должности ...,штраф,"лишение специального, воинского или почетного ...",ограничение свободы,NaN,необходимая оборона \n(ст. 37 УК РФ),крайняя необходимость\n(ст. 39 УК РФ),"обстоятельства, предусмотренные \nстатьями 38,..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,от лишения свободы,от иных мер или наказание не назначалось,NaN,NaN,NaN,NaN,NaN,NaN,NaN,на основании примечаний к статьям УК РФ (в свя...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,А,Б,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
4,Всего по составам УК РФ,1,658291,59,187794,169030,90,5,22966,187,56450,116058,795,183,85760,12629,1026,1370,3889,1446,2102,250,125866,18171,33329,10468,6182,8171,57475,7852,171,7080,38574,6,0,0


In [7]:
mainDF = deleteUnusedCols(mainDF, year)

In [8]:
mainDF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,Статьи УК РФ,ВСЕГО ОСУЖДЕНО,ВИДЫ основного НАКАЗАНИЯ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,По приговору освобождено осужденных от наказания,NaN,NaN,Оправдано,ПРЕКРАЩЕНО,NaN,NaN,NaN,NaN,NaN,NaN,Принудительные меры к невменяемым,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ,NaN,NaN,NaN,Преступление не является оконченным \n(пригото...,"Деяние совершено при обстоятельствах, исключаю...",NaN,NaN
1,NaN,NaN,пожизненное лишение свободы,лишение свободы,условное осуждение к лишению свободы,содержание в дисциплинарной воинской части,арест,ограничение свободы,ограничение по военной службе,исправительные работы,обязательные работы,принудительные работы,лишение права занимать определенные должности ...,штраф,условное осуждение к иным мерам,по амнистии,по другим основаниям:,NaN,NaN,"за отсутствием события, состава, непричастност...",по амнистии,за примирением с потерпевшим \n(ст. 25 УПК РФ),в связи с деятельным раскаянием \n(ст. 28 УПК...,назначена мера уголовно-правового характера су...,по другим основаниям,из графы 22,NaN,лишение права занимать определенные должности ...,штраф,"лишение специального, воинского или почетного ...",ограничение свободы,NaN,необходимая оборона \n(ст. 37 УК РФ),крайняя необходимость\n(ст. 39 УК РФ),"обстоятельства, предусмотренные \nстатьями 38,..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,от лишения свободы,от иных мер или наказание не назначалось,NaN,NaN,NaN,NaN,NaN,NaN,NaN,на основании примечаний к статьям УК РФ (в свя...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,А,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
4,Всего по составам УК РФ,658291,59,187794,169030,90,5,22966,187,56450,116058,795,183,85760,12629,1026,1370,3889,1446,2102,250,125866,18171,33329,10468,6182,8171,57475,7852,171,7080,38574,6,0,0


Удаляем строку "Всего по составам УК РФ" и строку с номерами колонок. Значения строки "Всего по составам УК РФ" сохраняем в отдельную переменную - они будут нужны позже, для проверки корректности обработки данных.

In [9]:
total_values = list(mainDF[mainDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][2:])
mainDF = dropFirstRows(mainDF, '105 ч. 1|105 ч.1|105ч. 1', 2)

In [10]:
mainDF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,Статьи УК РФ,ВСЕГО ОСУЖДЕНО,ВИДЫ основного НАКАЗАНИЯ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,По приговору освобождено осужденных от наказания,NaN,NaN,Оправдано,ПРЕКРАЩЕНО,NaN,NaN,NaN,NaN,NaN,NaN,Принудительные меры к невменяемым,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ,NaN,NaN,NaN,Преступление не является оконченным \n(пригото...,"Деяние совершено при обстоятельствах, исключаю...",NaN,NaN
1,NaN,NaN,пожизненное лишение свободы,лишение свободы,условное осуждение к лишению свободы,содержание в дисциплинарной воинской части,арест,ограничение свободы,ограничение по военной службе,исправительные работы,обязательные работы,принудительные работы,лишение права занимать определенные должности ...,штраф,условное осуждение к иным мерам,по амнистии,по другим основаниям:,NaN,NaN,"за отсутствием события, состава, непричастност...",по амнистии,за примирением с потерпевшим \n(ст. 25 УПК РФ),в связи с деятельным раскаянием \n(ст. 28 УПК...,назначена мера уголовно-правового характера су...,по другим основаниям,из графы 22,NaN,лишение права занимать определенные должности ...,штраф,"лишение специального, воинского или почетного ...",ограничение свободы,NaN,необходимая оборона \n(ст. 37 УК РФ),крайняя необходимость\n(ст. 39 УК РФ),"обстоятельства, предусмотренные \nстатьями 38,..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,от лишения свободы,от иных мер или наказание не назначалось,NaN,NaN,NaN,NaN,NaN,NaN,NaN,на основании примечаний к статьям УК РФ (в свя...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,105 ч. 1,5723,0,5683,28,0,0,1,0,0,0,0,0,0,0,0,9,2,19,1,0,0,0,0,59,0,448,6,0,3,532,834,1,0,0
6,105 ч. 2,1475,52,1416,3,0,0,1,0,0,0,0,0,0,0,0,3,0,20,1,0,0,0,0,10,0,99,0,0,1,977,315,0,0,0


Теперь разбиваем объединенные ячейки (если они есть), не теряя их содержимого: по каждому столбцу собираем текст, который распределен по объединенным ячейкам, в заголовок таблицы

In [11]:
mainDF = tackleMergedCells(mainDF)

/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [12]:
mainDF.head()

,Статьи УК РФ,ВСЕГО ОСУЖДЕНО,ВИДЫ основного НАКАЗАНИЯ пожизненное лишение свободы,ВИДЫ основного НАКАЗАНИЯ лишение свободы,ВИДЫ основного НАКАЗАНИЯ условное осуждение к лишению свободы,ВИДЫ основного НАКАЗАНИЯ содержание в дисциплинарной воинской части,ВИДЫ основного НАКАЗАНИЯ арест,ВИДЫ основного НАКАЗАНИЯ ограничение свободы,ВИДЫ основного НАКАЗАНИЯ ограничение по военной службе,ВИДЫ основного НАКАЗАНИЯ исправительные работы,ВИДЫ основного НАКАЗАНИЯ обязательные работы,ВИДЫ основного НАКАЗАНИЯ принудительные работы,ВИДЫ основного НАКАЗАНИЯ лишение права занимать определенные должности или заниматься определенной деятельностью,ВИДЫ основного НАКАЗАНИЯ штраф,ВИДЫ основного НАКАЗАНИЯ условное осуждение к иным мерам,По приговору освобождено осужденных от наказания по амнистии,По приговору освобождено осужденных от наказания по другим основаниям: от лишения свободы,По приговору освобождено осужденных от наказания от иных мер или наказание не назначалось,Оправдано,"ПРЕКРАЩЕНО за отсутствием события, состава, непричастностью к преступлению (п. 1, 2 ч. 1 ст. 24 УПК РФ, ч. 2 ст. 24 УПК РФ)",ПРЕКРАЩЕНО по амнистии,ПРЕКРАЩЕНО за примирением с потерпевшим (ст. 25 УПК РФ),ПРЕКРАЩЕНО в связи с деятельным раскаянием (ст. 28 УПК РФ),ПРЕКРАЩЕНО назначена мера уголовноправового характера судебный штраф (ст. 25.1 УПК РФ),ПРЕКРАЩЕНО по другим основаниям,ПРЕКРАЩЕНО из графы 22 на основании примечаний к статьям УК РФ (в связи с деятельным раскаянием ч. 2 ст. 28 УПК РФ),Принудительные меры к невменяемым,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ лишение права занимать определенные должности или заниматься определенной деятельностью,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ штраф,"ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ лишение специального, воинского или почетного звания, классного чина и государственных наград",ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ ограничение свободы,"Преступление не является оконченным (приготовление, покушение)","Деяние совершено при обстоятельствах, исключающих преступность необходимая оборона (ст. 37 УК РФ)","Деяние совершено при обстоятельствах, исключающих преступность крайняя необходимость (ст. 39 УК РФ)","Деяние совершено при обстоятельствах, исключающих преступность обстоятельства, предусмотренные статьями 38, 40 42 УК РФ"
0,Статьи УК РФ,ВСЕГО ОСУЖДЕНО,ВИДЫ основного НАКАЗАНИЯ пожизненное лишение ...,ВИДЫ основного НАКАЗАНИЯ лишение свободы,ВИДЫ основного НАКАЗАНИЯ условное осуждение к...,ВИДЫ основного НАКАЗАНИЯ содержание в дисципл...,ВИДЫ основного НАКАЗАНИЯ арест,ВИДЫ основного НАКАЗАНИЯ ограничение свободы,ВИДЫ основного НАКАЗАНИЯ ограничение по военн...,ВИДЫ основного НАКАЗАНИЯ исправительные работы,ВИДЫ основного НАКАЗАНИЯ обязательные работы,ВИДЫ основного НАКАЗАНИЯ принудительные работ...,ВИДЫ основного НАКАЗАНИЯ лишение права занима...,ВИДЫ основного НАКАЗАНИЯ штраф,ВИДЫ основного НАКАЗАНИЯ условное осуждение к...,По приговору освобождено осужденных от наказан...,По приговору освобождено осужденных от наказан...,По приговору освобождено осужденных от наказан...,Оправдано,"ПРЕКРАЩЕНО за отсутствием события, состава, не...",ПРЕКРАЩЕНО по амнистии,ПРЕКРАЩЕНО за примирением с потерпевшим (ст. 2...,ПРЕКРАЩЕНО в связи с деятельным раскаянием (с...,ПРЕКРАЩЕНО назначена мера уголовноправового ха...,ПРЕКРАЩЕНО по другим основаниям,ПРЕКРАЩЕНО из графы 22 на основании примечаний...,Принудительные меры к невменяемым,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ лишение права занимат...,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ штраф,"ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ лишение специального,...",ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ ограничение свободы,Преступление не является оконченным (приготовл...,"Деяние совершено при обстоятельствах, исключаю...","Деяние совершено при обстоятельствах, исключаю...","Деяние совершено при обстоятельствах, исключаю..."
1,NaN,NaN,пожизненное лишение свободы,лишение свободы,условное осуждение к лишению свободы,содержание в дисциплинарной воинской части,арест,ограничение свободы,ограничение по военной службе,исправительные работы,обязательные работы,принудительные работы,лишение права занимать определенные должно

Чистим заголовок таблицы от лишних пробелов, переносов строки и прочего, переводим в нижний регистр

In [13]:
mainDF = cleanСolsNames(mainDF)

In [14]:
mainDF.head()

,статьи ук рф,всего осуждено,виды основного наказания пожизненное лишение свободы,виды основного наказания лишение свободы,виды основного наказания условное осуждение к лишению свободы,виды основного наказания содержание в дисциплинарной воинской части,виды основного наказания арест,виды основного наказания ограничение свободы,виды основного наказания ограничение по военной службе,виды основного наказания исправительные работы,виды основного наказания обязательные работы,виды основного наказания принудительные работы,виды основного наказания лишение права занимать определенные должности или заниматься определенной деятельностью,виды основного наказания штраф,виды основного наказания условное осуждение к иным мерам,по приговору освобождено осужденных от наказания по амнистии,по приговору освобождено осужденных от наказания по другим основаниям от лишения свободы,по приговору освобождено осужденных от наказания от иных мер или наказание не назначалось,оправдано,"прекращено за отсутствием события, состава, непричастностью к преступлению (п. 1, 2 ч. 1 ст. 24 упк рф, ч. 2 ст. 24 упк рф)",прекращено по амнистии,прекращено за примирением с потерпевшим (ст. 25 упк рф),прекращено в связи с деятельным раскаянием (ст. 28 упк рф),прекращено назначена мера уголовноправового характера судебный штраф (ст. 25.1 упк рф),прекращено по другим основаниям,прекращено из графы 22 на основании примечаний к статьям ук рф (в связи с деятельным раскаянием ч. 2 ст. 28 упк рф),принудительные меры к невменяемым,дополнительное наказание лишение права занимать определенные должности или заниматься определенной деятельностью,дополнительное наказание штраф,"дополнительное наказание лишение специального, воинского или почетного звания, классного чина и государственных наград",дополнительное наказание ограничение свободы,"преступление не является оконченным (приготовление, покушение)","деяние совершено при обстоятельствах, исключающих преступность необходимая оборона (ст. 37 ук рф)","деяние совершено при обстоятельствах, исключающих преступность крайняя необходимость (ст. 39 ук рф)","деяние совершено при обстоятельствах, исключающих преступность обстоятельства, предусмотренные статьями 38, 40 42 ук рф"
0,Статьи УК РФ,ВСЕГО ОСУЖДЕНО,ВИДЫ основного НАКАЗАНИЯ пожизненное лишение ...,ВИДЫ основного НАКАЗАНИЯ лишение свободы,ВИДЫ основного НАКАЗАНИЯ условное осуждение к...,ВИДЫ основного НАКАЗАНИЯ содержание в дисципл...,ВИДЫ основного НАКАЗАНИЯ арест,ВИДЫ основного НАКАЗАНИЯ ограничение свободы,ВИДЫ основного НАКАЗАНИЯ ограничение по военн...,ВИДЫ основного НАКАЗАНИЯ исправительные работы,ВИДЫ основного НАКАЗАНИЯ обязательные работы,ВИДЫ основного НАКАЗАНИЯ принудительные работ...,ВИДЫ основного НАКАЗАНИЯ лишение права занима...,ВИДЫ основного НАКАЗАНИЯ штраф,ВИДЫ основного НАКАЗАНИЯ условное осуждение к...,По приговору освобождено осужденных от наказан...,По приговору освобождено осужденных от наказан...,По приговору освобождено осужденных от наказан...,Оправдано,"ПРЕКРАЩЕНО за отсутствием события, состава, не...",ПРЕКРАЩЕНО по амнистии,ПРЕКРАЩЕНО за примирением с потерпевшим (ст. 2...,ПРЕКРАЩЕНО в связи с деятельным раскаянием (с...,ПРЕКРАЩЕНО назначена мера уголовноправового ха...,ПРЕКРАЩЕНО по другим основаниям,ПРЕКРАЩЕНО из графы 22 на основании примечаний...,Принудительные меры к невменяемым,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ лишение права занимат...,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ штраф,"ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ лишение специального,...",ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ ограничение свободы,Преступление не является оконченным (приготовл...,"Деяние совершено при обстоятельствах, исключаю...","Деяние совершено при обстоятельствах, исключаю...","Деяние совершено при обстоятельствах, исключаю..."
1,NaN,NaN,пожизненное лишение свободы,лишение свободы,условное осуждение к лишению свободы,содержание в дисциплинарной воинской части,арест,ограничение свободы,ограничение по военной службе,исправительные работы,обязательные работы,принудительные работы,лишение права занимать определенные должнос

Переводим заголовок на английский. Английские названия для колонок лежат в отдельных файлах:    

- colNames2engNames.csv
- colNames2engNamesAdd.csv
- colNames2engNamesParameters.csv

Если колонка не была переименована, ее название печатается. Колонки с русскоязычными названиями далее будут удалены. В некоторых случаях так и должно быть: если колонка дублируется в исходных таблицах Судебного департамента, в своих данных мы используем ее только один раз.

In [15]:
columns = mainDF.columns # эта строка тоже будет нужна позже для проверки

columns2eng = pd.read_csv('colNames2engNames.csv')
columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
mainDF = renameColumns(mainDF, columns2eng)

In [16]:
mainDF.head()

,clause,totalConvicted,primaryLifeSentence,primaryImprisonment,primarySuspended,primaryMilitaryDisciplinaryUnit,primaryArrest,primaryRestrain,primaryRestrictionsInMilitaryService,primaryCorrectionalLabour,primaryCommunityService,primaryForcedLabour,primaryDisqualification,primaryFine,primaryOther,exemptionAmnesty,exemptionFromImprisonment,exemptionOther,acquittal,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalReconciliation,dismissalRepentance,dismissalCourtFine,dismissalOther,dismissalRepentance2,coerciveMeasures,addDisqualification,addFine,addTitlesWithdraw,addRestrain,unfinishedOffence,noCrimeSelf-defence,noCrimeNecessity,noCrimeOther
0,Статьи УК РФ,ВСЕГО ОСУЖДЕНО,ВИДЫ основного НАКАЗАНИЯ пожизненное лишение ...,ВИДЫ основного НАКАЗАНИЯ лишение свободы,ВИДЫ основного НАКАЗАНИЯ условное осуждение к...,ВИДЫ основного НАКАЗАНИЯ содержание в дисципл...,ВИДЫ основного НАКАЗАНИЯ арест,ВИДЫ основного НАКАЗАНИЯ ограничение свободы,ВИДЫ основного НАКАЗАНИЯ ограничение по военн...,ВИДЫ основного НАКАЗАНИЯ исправительные работы,ВИДЫ основного НАКАЗАНИЯ обязательные работы,ВИДЫ основного НАКАЗАНИЯ принудительные работ...,ВИДЫ основного НАКАЗАНИЯ лишение права занима...,ВИДЫ основного НАКАЗАНИЯ штраф,ВИДЫ основного НАКАЗАНИЯ условное осуждение к...,По приговору освобождено осужденных от наказан...,По приговору освобождено осужденных от наказан...,По приговору освобождено осужденных от наказан...,Оправдано,"ПРЕКРАЩЕНО за отсутствием события, состава, не...",ПРЕКРАЩЕНО по амнистии,ПРЕКРАЩЕНО за примирением с потерпевшим (ст. 2...,ПРЕКРАЩЕНО в связи с деятельным раскаянием (с...,ПРЕКРАЩЕНО назначена мера уголовноправового ха...,ПРЕКРАЩЕНО по другим основаниям,ПРЕКРАЩЕНО из графы 22 на основании примечаний...,Принудительные меры к невменяемым,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ лишение права занимат...,ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ штраф,"ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ лишение специального,...",ДОПОЛНИТЕЛЬНОЕ НАКАЗАНИЕ ограничение свободы,Преступление не является оконченным (приготовл...,"Деяние совершено при обстоятельствах, исключаю...","Деяние совершено при обстоятельствах, исключаю...","Деяние совершено при обстоятельствах, исключаю..."
1,NaN,NaN,пожизненное лишение свободы,лишение свободы,условное осуждение к лишению свободы,содержание в дисциплинарной воинской части,арест,ограничение свободы,ограничение по военной службе,исправительные работы,обязательные работы,принудительные работы,лишение права занимать определенные должности ...,штраф,условное осуждение к иным мерам,по амнистии,по другим основаниям:,NaN,NaN,"за отсутствием события, состава, непричастност...",по амнистии,за примирением с потерпевшим (ст. 25 УПК РФ),в связи с деятельным раскаянием (ст. 28 УПК РФ),назначена мера уголовноправового характера суд...,по другим основаниям,из графы 22,NaN,лишение права занимать определенные должности ...,штраф,"лишение специального, воинского или почетного ...",ограничение свободы,NaN,необходимая оборона (ст. 37 УК РФ),крайняя необходимость (ст. 39 УК РФ),"обстоятельства, предусмотренные статьями 38, 4..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,от лишения свободы,от иных мер или наказание не назначалось,NaN,NaN,NaN,NaN,NaN,NaN,NaN,на основании примечаний к статьям УК РФ (в свя...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,105 ч. 1,5723,0,5683,28,0,0,1,0,0,0,0,0,0,0,0,9,2,19,1,0,0,0,0,59,0,448,6,0,3,532,834,1,0,0
4,105 ч. 2,1475,52,1416,3,0,0,1,0,0,0,0,0,0,0,0,3,0,20,1,0,0,0,0,10,0,99,0,0,1,977,315,0,0,0


Вырезаем первую строку и строки, в которых остались ячейки с пропущенными значениями (NA). Это либо отработанные строки заголовка, либо "подвал" внизу таблицы

In [17]:
mainDF = dropNARows(mainDF)

In [18]:
mainDF.tail()

,clause,totalConvicted,primaryLifeSentence,primaryImprisonment,primarySuspended,primaryMilitaryDisciplinaryUnit,primaryArrest,primaryRestrain,primaryRestrictionsInMilitaryService,primaryCorrectionalLabour,primaryCommunityService,primaryForcedLabour,primaryDisqualification,primaryFine,primaryOther,exemptionAmnesty,exemptionFromImprisonment,exemptionOther,acquittal,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalReconciliation,dismissalRepentance,dismissalCourtFine,dismissalOther,dismissalRepentance2,coerciveMeasures,addDisqualification,addFine,addTitlesWithdraw,addRestrain,unfinishedOffence,noCrimeSelf-defence,noCrimeNecessity,noCrimeOther
921,360 ч. 2 \n(в редакции ФЗ от 08.12.2003 № 162-...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
922,361 ч. 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
923,361 ч. 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
924,361 ч. 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
925,Воинские преступления,1413,0,257,532,79,0,0,180,2,7,0,1,308,42,2,0,3,0,0,0,12,1,442,11,0,0,15,0,2,0,0,0,0,0


Из полученных таблиц мы удаляем строки, которые не несут информации о конкретных статьях:

- пустая
- Небольшой тяжести
- Средней тяжести
- Тяжкие
- Особо тяжкие
- Декриминализация
- пустые резервные строки

Показатели по этим строкам либо равны нулю, либо являются суммой уже учтенных статей (кроме строки Декриминализация).

Кроме того, в 2011 году нам пришлось удалить строку "Преступления, введенные в УК РФ после утверждения форм отчетности приказом № 115 от 10.06.2011" с одним осужденным к ограничению свободы, поскольку выяснить, по какой именно статье он был осужден, невозможно.

Еще в одном случае информацию о статье удалось восстановить, используя форму № 10-а «Отчет о числе осужденных по всем составам преступлений Уголовного кодекса Российской Федерации»:

| Год | Форма № 10.3 | Форма № 10-а |
| --- | --- | --- |
|2013|Резервная строка по разделу VIII "Преступления в сфере экономики" УК РФ (для составов преступлений, введенных в УК РФ после утверждения формы отчетности приказом № 130 от 28 июня 2013 г. )|185.6 ч.1-2|

Также в исходных данных есть строки, в которых объединены данные по нескольких статьям. За все года их нашлось шесть, детализированные данные по ним в Судебном департаменте нам предоставить не смогли. Там, где показатели равны нулю, такие строки мы разбиваем по принципу одна статья - одна строка, там, где разбить по статьям невозможно, оставляем, как есть, жертвуя принципом "одна статья - одна строка" ради полноты данных.

|Список строк, в которых объединены данные по нескольким статьм/частям|Год| Количество осужденных|
| --- | --- | --- |
|Составы преступлений, введенные в раздел X "Преступления против государственной власти" УК РФ после утверждения форм отчетности приказом № 127 ст.283.1; 303 ч.4; 330.1|2012| 0|
|Мошенничество, совершенное в сфере кредитования; при получении выплат; с использованием платежных карт; в сфере предпринимательской деят-ти; в сфере страхования; в сфере компьютерной информации (введ. ФЗ от 29.11.2012 N 207-ФЗ) ст. 159.1-159.6 |2012| 24|
|170.2 ч. 1; 170.2 ч. 2|2015| 0|
|284.1 ; 293 ч. 1.1|2015| 0|
|204 ч. 5, ч.6, ч.7, ч. 8 |2016| 46|
|159 ч. 5, ч. 6, ч.7| 2016| 13|

Также мы оставляем строку "Воинские преступления", в которой содержатся данные по статьям УК, связанным с военными преступлениями, поскольку в разбитом на статьи виде эти данные не дублируются. "Воинские преступления" есть в формах № 10.3 и № 10.3.1, и нет в № 10-а.

В 2018 году несколько статей отсутствуют в формах № 10.3 и № 10.3.1, но есть в форме № 10-а, поэтому чтобы все три таблицы соответствовали друг другу, мы вводим для этих статей пустые строки в формах № 10.3 и № 10.3.1. Исключение составляют части 3 и 4 статьи 215.3. Эта статья изменилась в 2018 году следующим образом: формально в ней появились новые части 4 и 5. Однако содержательно новыми являются части 1 и 2, а части 1, 2 и 3 старой редакции соответствуют частям 3, 4 и 5 новой редакции. Поскольку в таблицах Судебного департамента нет разделения частей этой статьи на новую и старую редакцию, а по новым частям 4 и 5 нет цифр, мы оставили статью 215.3 в старой редакции. То есть, в формах № 10.3 и № 10.3.1 мы добавлять новые части не стали, а в форме № 10-а удалили строки с частями 4 и 5.

In [19]:
mainDF = deleteUselessRows(mainDF, year)

In [20]:
clauses2Insert = {'2012': ['283.1', '303 ч.4', '330.1'], 
                  '2015': ['170.2 ч.1', '170.2 ч.2', '284.1', '293 ч.1.1'],
                  '2016': ['204.1 ч. 1', '204.1 ч. 2', '204.1 ч. 3', '204.1 ч. 4', '204.2 ч. 2'],
                  '2018': ['144.1', '253ч.3', '258.1ч.1.1', '258.1ч.2.1', '258.1ч.3.1', '315ч.1', '315ч.2']}

mainDF = insertEmptyRows(mainDF, year, clauses2Insert)

In [21]:
mainDF[mainDF['clause'].str.contains('283.1|303\s*ч.4|330.1|170.2\s*ч.1|170.2\s*ч.2|284.1|293\s*ч.1.1|204\s*ч.\s*5,\s*ч.6,\s*ч.7,\s*ч.\s*8|159\s*ч.\s*5,\s*ч.\s*6,\s*ч.7|159.1-159.6|144.1|215\.3ч\.\d', na=False)]

,acquittal,addDisqualification,addFine,addRestrain,addTitlesWithdraw,clause,coerciveMeasures,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalCourtFine,dismissalOther,dismissalReconciliation,dismissalRepentance,dismissalRepentance2,exemptionAmnesty,exemptionFromImprisonment,exemptionOther,noCrimeNecessity,noCrimeOther,noCrimeSelf-defence,primaryArrest,primaryCommunityService,primaryCorrectionalLabour,primaryDisqualification,primaryFine,primaryForcedLabour,primaryImprisonment,primaryLifeSentence,primaryMilitaryDisciplinaryUnit,primaryOther,primaryRestrain,primaryRestrictionsInMilitaryService,primarySuspended,totalConvicted,unfinishedOffence
742,0,0,0,0,0,283.1 ч. 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,4,0
743,0,0,0,0,0,283.1 ч. 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,4,0
745,0,0,0,0,0,284.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
900,0,0,0,0,0,330.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
923,0,0,0,0,0,144.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


При обработке файлов за 2016 год мы столкнулись с еще одной проблемой. Так, в форме № 10.3 за 2016 год приведены лишь аггрегированные данные по статьям 204.1 и 204.2 без указания статистики по отдельным частям этих статей. При этом в форме № 10.3.1 эти же статьи даны с детализацией по частям. Также и в форме № 10-а данные приведены с детализацией по частям.

Для того чтобы восстановить детализацию, отсутствующую в форме № 10.3, мы использовали информацию по числу осужденных по основной статье, которая содержится в форме № 10-а. Всего по статье 204.1 в 2016 году осудили трех человек: двоим присудили штраф, одному - условное лишение свободы. Из формы № 10-а мы узнаем, что по части 1 статьи 204.1 осудили одного человека, а по части 2 - двоих, по остальным же частям - 3 и 4 - никого не осудили. Из формы № 10.3.1 мы узнаем, что один человек получил штраф по части 1 статьи 204.1 и еще один - по части 2. Остается один осужденный по части 2 статьи 204.1, для которого нет других вариантов, кроме условного лишения свободы. Итого получаем, что по части 1 статьи 204.1 - одному человеку присудили штраф, по части 2 статьи 204.1 - одному штраф, второму - условное лишение свободы.

По статье 204.2 в 2016 году всего осудили 38 человек, прекратили 14 дел, освободили от наказания 3 человек. Как следует из формы № 10-а, все осужденные и все прекращенные дела приходятся на вторую часть этой статьи. Форма № 10-а не содержит информации об освобожденных от наказания. Мы также не нашли дел по части 2 статьи 204.2 в системе ГАС “Правосудие”, используя поиск по делам и судебным актам и полагаем, что прекращенные дела по 204.2 также относятся к первой части этой статьи. 

Поэтому строки, касающиеся этих статей мы переменовали и вставили недостающие, восстановив таким образом детализацию в форме № 10.3.

In [22]:
mainDF = fix204In2016(mainDF, year)

In [23]:
mainDF[mainDF['clause'].str.contains('204\.', na=False)]

,acquittal,addDisqualification,addFine,addRestrain,addTitlesWithdraw,clause,coerciveMeasures,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalCourtFine,dismissalOther,dismissalReconciliation,dismissalRepentance,dismissalRepentance2,exemptionAmnesty,exemptionFromImprisonment,exemptionOther,noCrimeNecessity,noCrimeOther,noCrimeSelf-defence,primaryArrest,primaryCommunityService,primaryCorrectionalLabour,primaryDisqualification,primaryFine,primaryForcedLabour,primaryImprisonment,primaryLifeSentence,primaryMilitaryDisciplinaryUnit,primaryOther,primaryRestrain,primaryRestrictionsInMilitaryService,primarySuspended,totalConvicted,unfinishedOffence
413,0,0,0,0,0,204.1 ч. 1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0
414,0,0,1,0,0,204.1 ч. 2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,6,2
415,0,0,1,0,0,204.1 ч. 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,1,4,0
416,0,0,0,0,0,204.1 ч. 4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
417,0,0,0,0,0,204.2 ч. 1,0,0,0,2,2,2,18,16,0,0,2,0,0,0,0,2,1,0,8,0,0,0,0,1,1,0,0,15,4
418,0,0,0,0,0,204.2 ч. 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Большое число статей в отчетных формах Судебного департамента содержат не просто номер статьи, но и примечание к ней.

In [24]:
mainDF[mainDF['clause'].str.contains('[а-яА-Я]{2,}', na=False)].head()

,acquittal,addDisqualification,addFine,addRestrain,addTitlesWithdraw,clause,coerciveMeasures,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalCourtFine,dismissalOther,dismissalReconciliation,dismissalRepentance,dismissalRepentance2,exemptionAmnesty,exemptionFromImprisonment,exemptionOther,noCrimeNecessity,noCrimeOther,noCrimeSelf-defence,primaryArrest,primaryCommunityService,primaryCorrectionalLabour,primaryDisqualification,primaryFine,primaryForcedLabour,primaryImprisonment,primaryLifeSentence,primaryMilitaryDisciplinaryUnit,primaryOther,primaryRestrain,primaryRestrictionsInMilitaryService,primarySuspended,totalConvicted,unfinishedOffence
10,0,0,0,0,0,"110 ч. 1, \n(включая ст. 110 старой редакции)",0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,1,0,6,14,3
29,259,3,0,2,0,115 ч. 1 \n(включая ст. 115 в старой редакции),35,1016,5,123,550,2356,24,0,0,0,46,0,0,3,0,783,361,0,1134,0,8,0,0,73,41,0,7,2453,2
31,6,0,0,0,0,116 ч. 1 \n(включая ст. 116 в старой редакции),1,61,0,13,47,189,2,0,0,0,0,0,0,0,0,92,59,0,51,0,3,0,0,5,1,0,1,212,0
39,0,0,0,0,0,118 ч. 3 \n(утратила силу - ФЗ от 08.12.2003 №...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,118 ч. 4 \n(утратила силу - ФЗ от 08.12.2003 №...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


В формах № 10.3 и № 10.3.1 все такие примечания мы удаляем, оставляя лишь номера статей и частей. В конечном датасете эти примечания не потеряются, так как мы возьмем их вместе с названиями статей из формы № 10-а.

In [25]:
mainDF = cleanClauseCol(mainDF) # чистим колонку с номерами статей от лишнего текста
mainDF = clauses2column(mainDF) # используя регулярное выражение, создаем колонки со номерами статей и частей 
mainDF = keepCombinedRows(mainDF, year) # переименовываем те номера статей, которые должны остаться объединенными, 
# т.к. они пострадали после предыдущей операции
firstCols = ['clause', 'part', 'totalConvicted']
mainDF = rearrangeCols(mainDF, firstCols) # меняем порядок столбцов для удобства
mainDF = sortTable(mainDF) # упорядочиваем строки

In [26]:
mainDF.head()

,clause,part,totalConvicted,acquittal,addDisqualification,addFine,addRestrain,addTitlesWithdraw,coerciveMeasures,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalCourtFine,dismissalOther,dismissalReconciliation,dismissalRepentance,dismissalRepentance2,exemptionAmnesty,exemptionFromImprisonment,exemptionOther,noCrimeNecessity,noCrimeOther,noCrimeSelf-defence,primaryArrest,primaryCommunityService,primaryCorrectionalLabour,primaryDisqualification,primaryFine,primaryForcedLabour,primaryImprisonment,primaryLifeSentence,primaryMilitaryDisciplinaryUnit,primaryOther,primaryRestrain,primaryRestrictionsInMilitaryService,primarySuspended,unfinishedOffence
0,105,105ч.1,5723,19,6,0,532,3,448,1,0,0,59,0,0,0,0,9,2,0,0,1,0,0,0,0,0,0,5683,0,0,0,1,0,28,834
1,105,105ч.2,1475,20,0,0,977,1,99,1,0,0,10,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1416,52,0,0,1,0,3,315
2,106,106,33,0,0,0,0,0,2,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,1,0,0,1,13,0,0,0,4,0,14,1
3,107,107ч.1,63,0,0,0,0,0,0,0,0,0,1,13,0,0,0,0,6,0,0,0,0,0,11,0,0,0,0,0,0,1,42,1,2,4
4,107,107ч.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


Выше мы сохранили значения строки "Всего по составам УК РФ", которая содержит сумммы по каждому столбцу. Теперь мы используем их для проверки корректности обработки данных, сравнив суммы по столбцам со значениями, указанными в строке "Всего по составам УК РФ".

К сожалению, как показала практика, числа эти не всегда совпадают, поэтому каждое несовпадение мы перепроверили вручную. Кроме того, еще в одном случае суммы действительно не совпадают из-за удаления в файлах 2011 год строки "Преступления, введенные в УК РФ после утверждения форм отчетности приказом № 115 от 10.06.2011" с одним осужденным к ограничению свободы (см. выше).

In [27]:
# выводит несоответствия сумм, если ничего не печатает - несоответствий нет
compareSums(mainDF, columns, total_values, 2)

<a id='10.3.1'></a>

### Обработка файлов формы № 10.3.1

Все описанные операции мы повторяем с файлами формы № 10.3.1, учитывая, что сама эта форма появляется лишь с 2011 года.

Есть и еще два несоотвествия форм № 10.3 и № 10.3.1:

- в 2012 году в основной таблице было зафиксирование появление частей у статьи 133, а в дополнительной - нет.

|№ 10.3|№ 10.3.1|
| --- | --- | 
|133 ч.1 (включая ст. 133 стар. ред., до ФЗ от 29.02.2012 № 14-ФЗ)|133|
|133 ч.2|133 ч.2|

- в 2013 и 2014 годах

|№ 10.3|№ 10.3.1|
| --- | --- | 
| 136 ч.1 | 136 ч.1 (утр. силу Федеральным законом от 07.12.2011 N 420-ФЗ )|
|136 ч.2, 136 (нов. ред.)| 136 (включая ч.2 ст.136 в ред. ФЗ от 07.12.2011 № 420-ФЗ)|

Чтобы формы соответствовали друг другу, мы приводим номера этих статей к тому виду, в котором они употребляются в форме № 10.3.1, исправляя в процессе обработки файлов 133 на 133ч.1 в файлах 2012 года и 136 на 136ч.2 в файлах 2013-2014 годов.

In [28]:
if year > 2010:
    parametersFile = 'ExcelFiles/MainParameters/' + str(year) + 'parameters.xls'
    parametersDF = pd.read_excel(parametersFile, 1, header=None)
    
    total_values = list(parametersDF[parametersDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][3:])
    
    parametersDF = dropFirstRows(parametersDF, 'Статьи УК РФ')
    parametersDF = deleteUnusedCols(parametersDF, year)
    parametersDF = dropFirstRows(parametersDF, '105 ч. 1|105 ч.1|105ч. 1', 2)
    
    parametersDF = tackleMergedCells(parametersDF)
    
    parametersDF = cleanСolsNames(parametersDF)
    columns2eng = pd.read_csv('colNames2engNamesParameters.csv')
    columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
    parametersDF = renameColumns(parametersDF, columns2eng)
    
    columns = parametersDF.columns
    
    parametersDF = parametersDF[parametersDF.columns[~parametersDF.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
    parametersDF = dropNARows(parametersDF)
    parametersDF = deleteUselessRows(parametersDF, year)
    
    clauses2Insert = {'2012': ['283.1', '303 ч.4', '330.1'], 
                      '2015': ['170.2 ч.1', '170.2 ч.2', '284.1', '293 ч.1.1'],
                     '2018': ['144.1', '253ч.3', '258.1ч.1.1', '258.1ч.2.1', '258.1ч.3.1', '315ч.1', '315ч.2']}
    
    parametersDF = insertEmptyRows(parametersDF, year, clauses2Insert)
    parametersDF = cleanClauseCol(parametersDF)
    
    parametersDF = solveProblem2012(parametersDF, year)
    
    parametersDF = clauses2column(parametersDF)
    parametersDF = solveProblem2013_2014(parametersDF, year)
    parametersDF = keepCombinedRows(parametersDF, year)
    
    firstCols = ['clause', 'part']
    parametersDF = rearrangeCols(parametersDF, firstCols)
    parametersDF = sortTable(parametersDF)

/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


лишение свободы всего (число лиц)
общая сумма штрафов (из гр. 13 "основное наказание" ф. 10.3) всего лиц
общая сумма штрафов (из гр. 28 "дополнительное наказание" ф. 10.3) всего лиц
общая сумма судебных штрафов, назначенных в соответствии со статьей 104.4 ук рф всего лиц


In [29]:
parametersDF[parametersDF['clause'].str.contains('133|136|144.1', na=False)]

,clause,part,addFine100_300,addFine1M,addFine25_100,addFine300_500,addFine5,addFine500_1M,addFine5_25,addFineSum,dismissalCourtFine100,dismissalCourtFine25_100,dismissalCourtFine5,dismissalCourtFine5_25,dismissalCourtFineSum,primaryFine100_300,primaryFine1M,primaryFine25_100,primaryFine300_500,primaryFine5,primaryFine500_1M,primaryFine5_25,primaryFineSum,primaryImprisonment1,primaryImprisonment10_15,primaryImprisonment15_20,primaryImprisonment1_2,primaryImprisonment2_3,primaryImprisonment3_5,primaryImprisonment5_8,primaryImprisonment8_10,primaryImprisonmentUnderLowerLimit
91,133,133ч.1,0,0,0,0,0,0,0,0,1,2,0,1,810000,0,0,1,0,0,0,1,75000,2,0,0,0,0,0,0,0,0
92,133,133ч.2,0,0,0,0,0,0,0,0,1,0,0,0,180000,0,0,0,0,0,0,0,0,1,0,0,6,3,1,0,0,0
112,136,136ч.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
113,136,136ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
140,144.1,144.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Снова проверяем суммы по столбцам.

In [30]:
compareSums(parametersDF, columns, total_values, 2)


Сумма значений в колонке dismissalCourtFine5_25 19095 
 Значение в строке "Всего по составам УК РФ"  19096

Сумма значений в колонке dismissalCourtFine25_100 4807 
 Значение в строке "Всего по составам УК РФ"  4806


<a id='10-a'></a>

### Обработка файлов формы № 10-а 

И теперь все то же самое для формы № 10-а, но с учетом еще нескольких нюансов.

1) В форме № 10-а есть колонка с именем статьи/части и примечанием, если она утратила силу или ее содержание существенно поменялось. Сам номер статьи/части при этом дается без примечаний.

| Год | Название статьи |Номер статьи |
| --- | --- | --- |
|Форма № 10-а|Понуждение к действиям сексуального характера (включая ст. 133 стар. ред., до ФЗ от 29.02.2012 № 14-ФЗ)|133 ч.1|
|Форма № 10.3| |133 ч.1 (включая ст. 133 стар. ред., до ФЗ от 29.02.2012 № 14-ФЗ)|

В нашем датасете мы использовали структуру формы № 10-а: примечания - в названиях статей/частей, номера статей/частей очищены от примечаний.

2) С 2017 года файл формы № 10-а разбит на несколько листов, так что их приходится обрабатывать по отдельности и склеивать в единую таблицу.

3) Форма № 10-а содержит большое количество сводных строк вида "ИТОГО ПО РАЗДЕЛУ VII. ПРЕСТУПЛЕНИЯ ПРОТИВ ЛИЧНОСТИ (сумма строк 3 - 7)", которые мы удаляем.

4) Исправляем те же несоотвествия записи статей 133 и 136, что и при обработке формы № 10.3.1, плюс маленькую опечатку в номере статье 282.2ч1.1 (нет точки после "ч").

Соответствия сумм в случае формы № 10-а мы не проверяем из-за невозможности подсчета числа осужденных по дополнительной квалификации: один человек может быть дополнительно осужден по нескольких дополнительным составам преступлений, из-за чего при сложении числа осужденных по всем статьям мы получим число, значительно превышающее реальное количество осужденных.

In [31]:
names = {'283.1': 'Незаконное получение сведений, составляющих государственную тайну',
         '303ч.4': 'Фальсификация результатов оперативно-разыскной деятельности лицом, уполномоченным на проведение оперативно-разыскных мероприятий, в целях уголовного преследования лица, заведомо непричастного к совершению преступления, либо в целях причинения вреда чести, достоинству и деловой репутации',
         '330.1': 'Злостное уклонение от исполнения обязанностей, определенных законодательством Российской Федерации о некоммерческих организациях, выполняющих функции иностранного агента',
         '144.1': 'Необоснованный отказ в приеме на работу или необоснованное увольнение лица, достигшего предпенсионного возраста',
         '253ч.3': 'Деяния, предусмотренные частями первой или второй настоящей статьи, совершенные лицом с использованием своего служебного положения либо группой лиц по предварительному сговору или организованной группой',
         '258.1ч.1.1': 'Незаконные приобретение или продажа особо ценных диких животных и водных биологических ресурсов, принадлежащих к видам, занесенным в Красную книгу Российской Федерации и (или) охраняемым международными договорами Российской Федерации, их частей и дериватов (производных) с использованием средств массовой информации либо электронных или информационно-телекоммуникационных сетей, в том числе сети "Интернет"',
         '258.1ч.2.1': 'Деяния, предусмотренные частью первой.1 настоящей статьи, совершенные должностным лицом с использованием своего служебного положения',
         '258.1ч.3.1': 'Деяния, предусмотренные частями первой.1 или второй.1 настоящей статьи, совершенные организованной группой',
         '315ч.1': 'Злостное неисполнение вступивших в законную силу приговора суда, решения суда или иного судебного акта, а равно воспрепятствование их исполнению лицом, подвергнутым административному наказанию за деяние, предусмотренное частью 4 статьи 17.15 Кодекса Российской Федерации об административных правонарушениях, совершенное в отношении того же судебного акта',
         '315ч.2': 'Злостное неисполнение представителем власти, государственным служащим, муниципальным служащим, а также служащим государственного или муниципального учреждения, коммерческой или иной организации вступивших в законную силу приговора суда, решения суда или иного судебного акта, а равно воспрепятствование их исполнению'
        }

addFile = 'ExcelFiles/Add/' + str(year) + 'add.xls'
if year >= 2017:
    df_list = pd.read_excel(addFile, sheet_name=None, header = None)
    dfs = []
    for key in list(df_list.keys())[2:-1]:
#     for key in list(df_list.keys())[5:6]:
        df = df_list[key]
        df = dropFirstRows(df, 'Виды преступлений')
        df = dropFirstRows(df, '[а-яА-Я]{3,}', 1)
        df = deleteUnusedCols(df, year)
        df = tackleMergedCells(df)
        columns2eng = pd.read_csv('colNames2engNamesAdd.csv')
        columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
        df = renameColumns(df, columns2eng) # тут заголовок таблицы предварительно чистить не надо

        df = df[df.columns[~df.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
        df = dropNARows(df)
        dfs.append(df)
    addDF = pd.concat(dfs, ignore_index=True)
else:
    addDF = pd.read_excel(addFile, 1, header=None)
    addDF = dropFirstRows(addDF, 'Виды преступлений')
    addDF = dropFirstRows(addDF, 'ВСЕГО ПО РАЗДЕЛАМ', 1)
    addDF = deleteUnusedCols(addDF, year)
    addDF = tackleMergedCells(addDF)

    columns2eng = pd.read_csv('colNames2engNamesAdd.csv')
    columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
    addDF = renameColumns(addDF, columns2eng) # тут заголовок таблицы предварительно чистить не надо

    addDF = addDF[addDF.columns[~addDF.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
    addDF = dropNARows(addDF)
addDF = deleteUselessRows(addDF, year)
clauses2Insert = {'2012': ['283.1', '303 ч.4', '330.1'],
                 '2018': ['258.1ч.2.1', '258.1ч.3.1']}
addDF = insertEmptyRows(addDF, year, clauses2Insert)
addDF = cleanClauseCol(addDF)
addDF = clauses2column(addDF)
addDF = nameSeparatedRows(addDF, names)

firstCols = ['name', 'clause', 'part', 'totalConvictedMain']
addDF = rearrangeCols(addDF, firstCols)

if year >= 2013:
    addDF.at[addDF.index[addDF['part'] == '136'][0], 'part'] = '136ч.2'
    addDF['part'] = addDF['part'].replace('168ч.1','168')
    addDF['part'] = addDF['part'].str.replace("282.2ч1.1", "282.2ч.1.1")
    
addDF = keepCombinedRows(addDF, year)
addDF = addMilitaryOfences(addDF)
addDF = addDF[~addDF['name'].str.contains('Составы преступлений, введенные в УК РФ в 2018 году')]

addDF['name'] = addDF['name'].str.replace('^\s{1}', '')
addDF['name'] = addDF['name'].str.replace('\n', '') 
addDF = sortTable(addDF)

/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().re

Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непри

In [32]:
addDF.tail()

,name,clause,part,totalConvictedMain,addAcquittalOffences,addAcquittalPersons,addDismissalOffences,addDismissalOtherOffences,addDismissalOtherPersons,addDismissalPersons,addTotalOffences,addTotalPersons,addUnfitToPleadOffences,addUnfitToPleadPersons
925,"Нападение на лиц или учреждения, пользующиеся ...",360,360ч.2,0,0,0,0,0,0,0,0,0,0,0
926,"Совершение вне пределов территории РФ взрыва, ...",361,361ч.1,0,0,0,0,0,0,0,0,0,0,0
927,"Акт международного терроризма, финансирование ...",361,361ч.2,0,0,0,0,0,0,0,0,0,0,0
928,"Деяния, предусмотренные частью первой настояще...",361,361ч.3,0,0,0,0,0,0,0,0,0,0,0
929,Воинские преступления,Воинские преступления,Воинские преступления,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id='data_check'></a>

### Проверка корректности обработки данных

Теперь нужно соединить все три таблицы за год в одну, но сначала проведем несколько проверок.

1) все три таблицы должны быть одной длины.

In [33]:
if year > 2010:
    checkTablesLen(mainDF, addDF, parametersDF)
else:
    checkTablesLen(mainDF, addDF)

ОК: длины таблиц № 10.3 и № 10.3.1 совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают


2) номера статей при построчном сравнении должны совпадать. Однако простое построчное сравнение может нас обмануть.В отчетных формах Судебного департамента могут соседствовать актуальная и утратившая силу части одной и той же статьи с одинаковыми номерами, при этом все примечания к номерам статей мы убрали, оставив их у названий статей/частей. Поэтому построчно имеет смысл сравнивать не только колонку с номерами статей, но и числа в тех столбцах, которые в разных формах должны содержать одинаковую информацию.


Так в форме № 10.3 и № 10.3.1 должны совпадать соответственно колонки "Лишение свободы" и сумма чисел из колонок "До 1 года вкл.", "Свыше 1 до 2 лет вкл.",	"Свыше 2 до 3 лет вкл.", "Свыше 3 до 5 лет вкл.", "Свыше 5 до 8 лет вкл.", "Свыше 8 до 10 лет вкл.", "Свыше 10 до 15 лет вкл.", "Свыше 15 до 20 лет вкл.".

А в формах № 10.3 и № 10-а - колонки "ВСЕГО ОСУЖДЕНО ЛИЦ" и "Число осужденных лиц по основной статье".

На практике так происходит не всегда и числа регулярно расходятся, поэтому приходится ориентироваться не на их полное совпадение, а на величину разницы между ними.

In [34]:
if year > 2010:
    checkNumbersBetweenForms(year, mainDF, addDF, parametersDF)
else:
    checkNumbersBetweenForms(year, mainDF, addDF)

Не совпадает число осужденных по основной статье. 
Статья:  253ч.3 
10.3: 0 
10-a: 2 
Год: 2018
Не совпадает число осужденных по основной статье. 
Статья:  Воинские преступления 
10.3: 1413 
10-a: nan 
Год: 2018


Теперь можно свести все таблицы за один год в одну, удалив дублирующиеся столбцы, а именно столбцы с номером и частью статьи в формах № 10.3.1 и № 10-а и столбцы с числом осужденных по основному составу преступления в форме № 10-а.

In [35]:
if year > 2010:
    addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
    parametersDF = parametersDF.drop(['clause', 'part'], axis=1)
    df = pd.concat([mainDF, parametersDF, addDF], axis=1)
else:
    addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
    df = pd.concat([mainDF, addDF], axis=1)

firstCols = ['name', 'clause', 'part', 'totalConvicted']
df = rearrangeCols(df, firstCols)

In [36]:
df.head()

,name,clause,part,totalConvicted,acquittal,addDisqualification,addFine,addRestrain,addTitlesWithdraw,coerciveMeasures,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalCourtFine,dismissalOther,dismissalReconciliation,dismissalRepentance,dismissalRepentance2,exemptionAmnesty,exemptionFromImprisonment,exemptionOther,noCrimeNecessity,noCrimeOther,noCrimeSelf-defence,primaryArrest,primaryCommunityService,...,primaryFine25_100,primaryFine300_500,primaryFine5,primaryFine500_1M,primaryFine5_25,primaryFineSum,primaryImprisonment1,primaryImprisonment10_15,primaryImprisonment15_20,primaryImprisonment1_2,primaryImprisonment2_3,primaryImprisonment3_5,primaryImprisonment5_8,primaryImprisonment8_10,primaryImprisonmentUnderLowerLimit,addAcquittalOffences,addAcquittalPersons,addDismissalOffences,addDismissalOtherOffences,addDismissalOtherPersons,addDismissalPersons,addTotalOffences,addTotalPersons,addUnfitToPleadOffences,addUnfitToPleadPersons
0,Убийство,105,105ч.1,5723,19,6,0,532,3,448,1,0,0,59,0,0,0,0,9,2,0,0,1,0,0,...,0,0,0,0,0,0,1,711,0,6,26,186,2546,2207,181,7,7,3,1,1,3,77,72,448,448
1,Убийство при отягчающих обстоятельствах,105,105ч.2,1475,20,0,0,977,1,99,1,0,0,10,0,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,1,568,411,0,0,36,157,243,35,42,33,9,4,2,9,157,38,99,99
2,Убийство матерью новорожденного ребенка,106,106,33,0,0,0,0,0,2,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2
3,"Убийство, совершенное в состоянии аффекта",107,107ч.1,63,0,0,0,0,0,0,0,0,0,1,13,0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
4,Убийство в состоянии аффекта двух или более лиц,107,107ч.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


И привести таблицу в длинную форму.

In [37]:
dfMelted = meltTable(df, year)

In [38]:
dfMelted.head()

,year,clause,part,name,parameter,value
0,2018,105,105ч.1,Убийство,totalConvicted,5723
1,2018,105,105ч.2,Убийство при отягчающих обстоятельствах,totalConvicted,1475
2,2018,106,106,Убийство матерью новорожденного ребенка,totalConvicted,33
3,2018,107,107ч.1,"Убийство, совершенное в состоянии аффекта",totalConvicted,63
4,2018,107,107ч.2,Убийство в состоянии аффекта двух или более лиц,totalConvicted,1


<a id='all'></a>

Используя приведенный выше код, мы обрабатываем файлы за все доступные года и получаем финальную таблицу.

In [39]:
names = {'283.1': 'Незаконное получение сведений, составляющих государственную тайну',
         '303ч.4': 'Фальсификация результатов оперативно-разыскной деятельности лицом, уполномоченным на проведение оперативно-разыскных мероприятий, в целях уголовного преследования лица, заведомо непричастного к совершению преступления, либо в целях причинения вреда чести, достоинству и деловой репутации',
         '330.1': 'Злостное уклонение от исполнения обязанностей, определенных законодательством Российской Федерации о некоммерческих организациях, выполняющих функции иностранного агента',
         '144.1': 'Необоснованный отказ в приеме на работу или необоснованное увольнение лица, достигшего предпенсионного возраста',
         '215.3ч.1': 'Самовольное подключение к нефтепроводам, нефтепродуктопроводам и газопроводам, совершенное лицом, подвергнутым административному наказанию за аналогичное деяние',
         '215.3ч.2': 'Деяние, предусмотренное частью первой настоящей статьи, совершенное в отношении магистральных трубопроводов',
         '215.3ч.3': 'Разрушение, повреждение или приведение иным способом в негодное для эксплуатации состояние нефтепроводов, нефтепродуктопроводов, газопроводов, а также технологически связанных с ними объектов, сооружений, средств связи, автоматики, сигнализации, которые повлекли или могли повлечь нарушение их нормальной работы и были совершены из корыстных или хулиганских побуждений',
         '215.3ч.4': 'Деяния, предусмотренные частью третьей настоящей статьи, совершенные а) группой лиц по предварительному сговору; б) в отношении магистральных трубопроводов',
         '215.3ч.5': 'Деяния, предусмотренные частями первой - четвертой настоящей статьи, повлекшие по неосторожности смерть человека или иные тяжкие последствия',
         '253ч.3': 'Деяния, предусмотренные частями первой или второй настоящей статьи, совершенные лицом с использованием своего служебного положения либо группой лиц по предварительному сговору или организованной группой',
         '258.1ч.1.1': 'Незаконные приобретение или продажа особо ценных диких животных и водных биологических ресурсов, принадлежащих к видам, занесенным в Красную книгу Российской Федерации и (или) охраняемым международными договорами Российской Федерации, их частей и дериватов (производных) с использованием средств массовой информации либо электронных или информационно-телекоммуникационных сетей, в том числе сети "Интернет"',
         '258.1ч.2.1': 'Деяния, предусмотренные частью первой.1 настоящей статьи, совершенные должностным лицом с использованием своего служебного положения',
         '258.1ч.3.1': 'Деяния, предусмотренные частями первой.1 или второй.1 настоящей статьи, совершенные организованной группой',
         '315ч.1': 'Злостное неисполнение вступивших в законную силу приговора суда, решения суда или иного судебного акта, а равно воспрепятствование их исполнению лицом, подвергнутым административному наказанию за деяние, предусмотренное частью 4 статьи 17.15 Кодекса Российской Федерации об административных правонарушениях, совершенное в отношении того же судебного акта',
         '315ч.2': 'Злостное неисполнение представителем власти, государственным служащим, муниципальным служащим, а также служащим государственного или муниципального учреждения, коммерческой или иной организации вступивших в законную силу приговора суда, решения суда или иного судебного акта, а равно воспрепятствование их исполнению'
        }
dfList = []
for year in range(2009, 2019):
    mailFile = 'ExcelFiles/Main/' + str(year) + 'main.xls'
    mainDF = pd.read_excel(mailFile, 1, header=None)
    mainDF = dropFirstRows(mainDF, 'Статьи УК РФ')
    mainDF = deleteUnusedCols(mainDF, year)
    total_values = list(mainDF[mainDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][2:])
    mainDF = dropFirstRows(mainDF, '105 ч. 1|105 ч.1|105ч. 1', 2)
    mainDF = tackleMergedCells(mainDF)
    mainDF = cleanСolsNames(mainDF)
    columns2eng = pd.read_csv('colNames2engNames.csv')
    columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
    mainDF = renameColumns(mainDF, columns2eng)
    mainDF = dropNARows(mainDF)
    mainDF = deleteUselessRows(mainDF, year)
    clauses2Insert = {'2012': ['283.1', '303 ч.4', '330.1'], 
                      '2015': ['170.2 ч.1', '170.2 ч.2', '284.1', '293 ч.1.1'],
                      '2016': ['204.1 ч. 1', '204.1 ч. 2', '204.1 ч. 3', '204.1 ч. 4', '204.2 ч. 2'],
                      '2018': ['144.1', '215.3ч.4', '215.3ч.5', '253ч.3', '258.1ч.1.1', '258.1ч.2.1', '258.1ч.3.1', '315ч.1', '315ч.2']}

    mainDF = insertEmptyRows(mainDF, year, clauses2Insert)
    mainDF = fix204In2016(mainDF, year)
    mainDF = cleanClauseCol(mainDF)
    mainDF = clauses2column(mainDF)
    mainDF = keepCombinedRows(mainDF, year)
    firstCols = ['clause', 'part', 'totalConvicted']
    mainDF = rearrangeCols(mainDF, firstCols)
    mainDF = sortTable(mainDF)

    if year > 2010:
        parametersFile = 'ExcelFiles/MainParameters/' + str(year) + 'parameters.xls'
        parametersDF = pd.read_excel(parametersFile, 1, header=None)

        total_values = list(parametersDF[parametersDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][3:])

        parametersDF = dropFirstRows(parametersDF, 'Статьи УК РФ')
        parametersDF = deleteUnusedCols(parametersDF, year)
        parametersDF = dropFirstRows(parametersDF, '105 ч. 1|105 ч.1|105ч. 1', 2)

        parametersDF = tackleMergedCells(parametersDF)

        parametersDF = cleanСolsNames(parametersDF)
        columns2eng = pd.read_csv('colNames2engNamesParameters.csv')
        columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
        parametersDF = renameColumns(parametersDF, columns2eng)

        columns = parametersDF.columns

        parametersDF = parametersDF[parametersDF.columns[~parametersDF.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
        parametersDF = dropNARows(parametersDF)
        parametersDF = deleteUselessRows(parametersDF, year)

        clauses2Insert = {'2012': ['283.1', '303 ч.4', '330.1'], 
                          '2015': ['170.2 ч.1', '170.2 ч.2', '284.1', '293 ч.1.1'],
                          '2018': ['144.1', '215.3ч.4', '215.3ч.5', '253ч.3', '258.1ч.1.1', '258.1ч.2.1', '258.1ч.3.1', '315ч.1', '315ч.2']}

        parametersDF = insertEmptyRows(parametersDF, year, clauses2Insert)
        parametersDF = cleanClauseCol(parametersDF)

        parametersDF = solveProblem2012(parametersDF, year)

        parametersDF = clauses2column(parametersDF)
        parametersDF = solveProblem2013_2014(parametersDF, year)
        parametersDF = keepCombinedRows(parametersDF, year)

        firstCols = ['clause', 'part']
        parametersDF = rearrangeCols(parametersDF, firstCols)
        parametersDF = sortTable(parametersDF)
    
    addFile = 'ExcelFiles/Add/' + str(year) + 'add.xls'
    if year >= 2017:
        df_list = pd.read_excel(addFile, sheet_name=None, header = None)
        dfs = []
        for key in list(df_list.keys())[2:-1]:
            df = df_list[key]
            df = dropFirstRows(df, 'Виды преступлений')
            df = dropFirstRows(df, '[а-яА-Я]{3,}', 1)
            df = deleteUnusedCols(df, year)
            df = tackleMergedCells(df)
            columns2eng = pd.read_csv('colNames2engNamesAdd.csv')
            columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
            df = renameColumns(df, columns2eng) # тут заголовок таблицы предварительно чистить не надо

            df = df[df.columns[~df.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
            df = dropNARows(df)
            dfs.append(df)
        addDF = pd.concat(dfs, ignore_index=True)
    else:
        addDF = pd.read_excel(addFile, 1, header=None)
        addDF = dropFirstRows(addDF, 'Виды преступлений')
        addDF = dropFirstRows(addDF, 'ВСЕГО ПО РАЗДЕЛАМ', 1)
        addDF = deleteUnusedCols(addDF, year)
        addDF = tackleMergedCells(addDF)

        columns2eng = pd.read_csv('colNames2engNamesAdd.csv')
        columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
        addDF = renameColumns(addDF, columns2eng) # тут заголовок таблицы предварительно чистить не надо

        addDF = addDF[addDF.columns[~addDF.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
        addDF = dropNARows(addDF)
    addDF = deleteUselessRows(addDF, year)
    clauses2Insert = {'2012': ['283.1', '303 ч.4', '330.1'],
                     '2018': ['258.1ч.2.1', '258.1ч.3.1']}
    addDF = insertEmptyRows(addDF, year, clauses2Insert)
    addDF = cleanClauseCol(addDF)
    addDF = clauses2column(addDF)
    addDF = nameSeparatedRows(addDF, names)

    firstCols = ['name', 'clause', 'part', 'totalConvictedMain']
    addDF = rearrangeCols(addDF, firstCols)

    if year >= 2013:
        addDF.at[addDF.index[addDF['part'] == '136'][0], 'part'] = '136ч.2'
        addDF['part'] = addDF['part'].replace('168ч.1','168')
        addDF['part'] = addDF['part'].str.replace("282.2ч1.1", "282.2ч.1.1")

    addDF = keepCombinedRows(addDF, year)
    addDF = addMilitaryOfences(addDF)
    addDF = addDF[~addDF['name'].str.contains('Составы преступлений, введенные в УК РФ в 2018 году')]
    addDF['name'] = addDF['name'].str.replace('^\s{1}', '')
    addDF['name'] = addDF['name'].str.replace('\n', '') 
    addDF = sortTable(addDF)
           
    if year > 2010:
        addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
        parametersDF = parametersDF.drop(['clause', 'part'], axis=1)
        df = pd.concat([mainDF, parametersDF, addDF], axis=1)
    else:
        addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
        df = pd.concat([mainDF, addDF], axis=1)

    firstCols = ['name', 'clause', 'part', 'totalConvicted']
    df = rearrangeCols(df, firstCols)

    dfMelted = meltTable(df, year)
    
    dfList.append(dfMelted)

df_final = pd.concat(dfList)
df_final.reset_index(drop=True, inplace=True)
df = df_final

дополнительное наказание  лишение права занимать опр. должности или заниматься опр. деятельностью
дополнительное наказание  штраф
дополнительное наказание  лишение специального, воинского или почетного звания, классного чина и государственных наград


/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


по приговору освобождено осужденных от наказания по другим основаниям  от иных мер или наказание не назначалось
дополнительное наказание  лишение права занимать опр. должности или заниматься опр. деятельностью
дополнительное наказание  штраф
дополнительное наказание  лишение специального, воинского или почетного звания, классного чина и государственных наград
лишение свободы всего
по приговору освобождено осужденных от наказания по другим основаниям  от иных мер или наказание не назначалось
дополнительное наказание  лишение права занимать опр. должности или заниматься опр. деятельностью
дополнительное наказание  штраф
дополнительное наказание  лишение специального, воинского или почетного звания, классного чина и государственных наград
лишение свободы всего
по приговору освобождено осужденных от наказания по другим основаниям  от иных мер или наказание не назначалось
дополнительное наказание  лишение права занимать опр. должности или заниматься опр. деятельностью
дополнительное наказан

/home/kate/sudstat/venv/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непри

Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непри